## Francony Alexandre

# WebScrapper for MTG decks on cardmarket.com

Nous allons dans un premier temps nous assurer que le site est atteignable à l'aide de la librairie requests et de la fonction get_status_code. Cette fonction sera réutilisée dans le code pour vérifier que le site est toujours atteignable avant chaque étape de récupération majeure.

In [2]:
import requests
import csv
import os
import requests
from bs4 import BeautifulSoup
import re

def get_status_code(url):
    response = requests.get(url)
    return response.status_code

url = "https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One"
status_code = get_status_code(url)
print("Le code d'état de Cardmarket est", status_code)


Le code d'état de Cardmarket est 200


In [3]:
def prep(card):
    #transformer tout les caractères non alphanumériques en tirets et tout en minuscule et les espaces en tirets
    return re.sub(r'\W+', '-', card).lower()

In [4]:
with open('data/deck.txt', 'r') as file:
    decklist = file.read()

#Récupérer la première ligne du fichier qui contient 'MIN_QUALITY = "<qualité>"'
MIN_QUALITY = decklist.splitlines()[0].split(" = ")[1].strip('"')
#print(MIN_QUALITY)

#Récupérer chaque autre ligne du fichier qui contient "quantité nom de la carte" et les stocker dans un dictionaire
deck = {}
for line in decklist.splitlines()[1:]:
    quantity, card = line.split(" ", 1)
    deck[card] = quantity
#print(deck)

#et maintenant ajoutons l'url de chaque carte dans le dictionnaire
for card in deck:
    deck[card] = [deck[card], "https://www.cardmarket.com/en/Magic/Cards/" + prep(card)]
#print(deck)


In [5]:
qualities = []
def switch_case(argument):
    switcher = {
        "MT": ["MT"],
        "NM": ["NM", "MT"],
        "EX": ["EX", "NM", "MT"],
        "GD": ["GD", "EX", "NM", "MT"],
        "LP": ["LP", "GD", "EX", "NM", "MT"],
        "PL": ["PL", "LP", "GD", "EX", "NM", "MT"],
        "PO": ["PO", "PL", "LP", "GD", "EX", "NM", "MT"],
    }
    return switcher.get(argument, ["MT"])

qualities = switch_case(MIN_QUALITY)
#print(qualities)


In [6]:
def getCardName(card):
    #récupérer le nom de la carte qui est la clé du dictionnaire
    card_name = card
    if(card_name == ''):
        card_name = 'Undefined'
    return card_name;

def getCardPrices(card):
    cards_sales = []
    tab_prices = []
    
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find_all('div', {'class': 'row no-gutters article-row'})

    for row in table:
        cards_sales.append(row)

    #récupérer les 5 premiers prix de vente de la carte
    for card in cards_sales:
        card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
        tab_prices.append(card_price.replace(',', '.'))
        if len(tab_prices) == 5:
            break

    return tab_prices;

def getCardQuantity(card):
    # récupérer l'url de la carte
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération de la quantité de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]

    # récupérer seulement la quantité, qui est contenu dans le premier élément du tableau
    card_quantity = frame_infos[0].text.strip()

    return card_quantity;

def getCardAveragePrice(card):
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération des données en chiffre de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]
    
    # récupérer seulement le prix moyen sur 7 jours, qui est contenu dans l'avant dernier élément du tableau
    card_average_price = frame_infos[-2].text.strip()
    return card_average_price.replace(',', '.').replace(' €', '').replace('£', '');

def getBuyableInfo(card):
    cards_sales = {}

    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find('div', {'class': 'table-body'})

    #dans table, trouver chaque ligne représentant une carte à vendre avec son prix, sa qualité et sa langue
    for row in table:
        cards_sales[i] = [row.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2], 
                          row.find('span', {"class" : "badge"}).text.strip(), 
                          row.find_all('span', {'class': 'icon mr-2'})[1].get('data-original-title')]
        i=i+1

    return cards_sales

In [10]:
cards_sales = {}

# Récupération du contenu de la page
url = deck["Jodah, the Unifier"][1]
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Recherche de la liste des cartes
table = soup.find('div', {'class': 'table-body'})

# dans le dictionaire cards_sales, créer une clé avec le contenu de la row, et en valeur le prix de la carte, sa qualité et sa langue
i = 0
for row in table:
    cards_sales[row] = [i,
                        row.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2], 
                        row.find('span', {"class" : "badge"}).text.strip(), 
                        row.find_all('span', {'class': 'icon mr-2'})[1].get('data-original-title')]
    i=i+1

print(cards_sales)

# prendre le dernier élément de card.find_all('span', {'class' : 'icon mr-2'})
# prendre le span['data-original-title'] de ce dernier élément


ConnectionError: HTTPSConnectionPool(host='www.cardmarket.com', port=443): Max retries exceeded with url: /en/Magic/Cards/jodah-the-unifier (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016E22D9A510>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [8]:
print(getBuyableInfo('Jodah, the Unifier'))

TypeError: 'NoneType' object is not subscriptable

A présent que nous avons pu créer notre nouveau dictionnaire avec les cartes sélectionnées, nous allons pouvoir créer un nouveau fichier csv qui contiendra uniquement les infos des cartes sélectionnées et avoir plus rapidement les informations afin de pouvoir réagir presque en temps réel. Les cartes que j'ai sélectionnés sont purement arbitraire, je prévois de les inclure dans l'un de mes decks.

In [ ]:
def getCardInfoWL(card):
    url = deck[card][1]
    if(get_status_code(url) == 200):
        card_intel = []
        card_intel.append(getCardName(card))
        card_intel.append(getCardPrices(card))
        card_intel.append(getCardQuantity(card))
        card_intel.append(getCardAveragePrice(card))
        card_intel.append(getBuyableInfo(card))

    else:
        print("Le code d'état de la page de ", card, " est ", get_status_code(url), ", la page n'a pas pu être chargée.")
    return card_intel

#for card in card_url:
#    print(getCardInfoWL(card))

In [ ]:
total_price = 0

for cards in deck:
    

In [ ]:
# à décommenter pour créer le fichier .csv (prends environ 1m20 max)
# Création du dossier s'il n'existe pas déjà
if not os.path.exists('data'):
    os.makedirs('data')

# Création du fichier .csv
with open('data/wants.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    # nommer les collonnes du tableau
    writer.writerow(['Card Name', 'Prices', 'Quantity', 'Average Price (7 days)'])
    for card in card_url:
        writer.writerow(getCardInfoWL(card))